In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import time

In [60]:
import pandas as pd
df = pd.read_csv("Twitter.csv",sep="\t")

In [61]:
df.head()

,clean_tweet,airline_sentiment
0,virginamerica plus added commercials experienc...,positive
1,virginamerica really aggressive blast obnoxiou...,negative
2,virginamerica really big bad thing,negative
3,virginamerica seriously would pay flight seats...,negative
4,virginamerica yes nearly every time fly vx ear...,positive


In [62]:
df = df[:501]

In [63]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['airline_sentiment'])

df['airline_sentiment'] = le.transform(df['airline_sentiment'])
df.head()

,clean_tweet,airline_sentiment
0,virginamerica plus added commercials experienc...,1
1,virginamerica really aggressive blast obnoxiou...,0
2,virginamerica really big bad thing,0
3,virginamerica seriously would pay flight seats...,0
4,virginamerica yes nearly every time fly vx ear...,1


In [64]:
x = df.clean_tweet
y = df.airline_sentiment

In [65]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.20, random_state=42)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

400 400
101 101


In [66]:
df['clean_tweet'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)

virginamerica     339.0
plus                3.0
added               3.0
commercials         1.0
experience         14.0
                  ...  
planning            1.0
completed           1.0
doubt               1.0
vacation            1.0
responsibility      1.0
Length: 1756, dtype: float64

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(x_train)

CountVectorizer()

In [68]:
x_train_dtm = vect.transform(x_train).toarray()
x_test_dtm = vect.transform(x_test).toarray()

In [69]:
from sklearn.svm import SVC
model = SVC(kernel='linear', random_state = 10)
model.fit(x_train_dtm, y_train)
pred = model.predict(x_test_dtm)

In [70]:
accuracy_score(y_test,pred)

0.8910891089108911

In [71]:
from sklearn.decomposition import KernelPCA
kernel_pca_rbf = KernelPCA(n_components=2, kernel="rbf")
kernel_pca_rbf.fit(x_train_dtm)
train_features_rbf = kernel_pca_rbf.transform(x_train_dtm)
test_features_rbf = kernel_pca_rbf.transform(x_test_dtm)

In [72]:
!pip install qiskit==0.45.2

In [73]:
from qiskit.circuit.library import PauliFeatureMap
num_features = train_features_rbf.shape[1]
feature_map = PauliFeatureMap(feature_dimension=num_features, reps=1, entanglement = 'linear',
insert_barriers =True)

In [74]:
feature_map.draw()

┌─────────────────────────────┐
q_0: ┤0                            ├
     │  PauliFeatureMap(x[0],x[1]) │
q_1: ┤1                            ├
     └─────────────────────────────┘

In [75]:
print(feature_map.decompose().decompose().depth())

5


In [76]:
feature_map.decompose().draw()

┌───┐ ░ ┌─────────────┐                                          
q_0: ┤ H ├─░─┤ P(2.0*x[0]) ├──■────────────────────────────────────■──
     ├───┤ ░ ├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐
q_1: ┤ H ├─░─┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├
     └───┘ ░ └─────────────┘└───┘└──────────────────────────────┘└───┘

In [77]:
!pip install qiskit-aer

In [78]:
from qiskit import Aer, execute
backend = Aer.get_backend("qasm_simulator")

In [79]:
!pip install qiskit_machine_learning

In [80]:
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.algorithms import QSVC

sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

## QSVC

In [81]:
qsvc = QSVC(quantum_kernel=kernel)

qsvc.fit(train_features_rbf,y_train)
qsvc_score = qsvc.score(test_features_rbf, y_test)

print(f"QSVC classification test score: {qsvc_score}")

QSVC classification test score: 0.6336633663366337


##VQC

In [85]:
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.algorithms.optimizers import ISRES, ADAM, COBYLA
from qiskit.primitives import Sampler

In [89]:
from qiskit.circuit.library import RealAmplitudes
ansatz = RealAmplitudes(num_qubits=num_features, reps=1)
ansatz.decompose().draw(fold=20)

┌──────────┐»
q_0: ┤ Ry(θ[0]) ├»
     ├──────────┤»
q_1: ┤ Ry(θ[1]) ├»
     └──────────┘»
«          »
«q_0: ──■──»
«     ┌─┴─┐»
«q_1: ┤ X ├»
«     └───┘»
«     ┌──────────┐
«q_0: ┤ Ry(θ[2]) ├
«     ├──────────┤
«q_1: ┤ Ry(θ[3]) ├
«     └──────────┘

In [92]:
from matplotlib import pyplot as plt
from IPython.display import clear_output

objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)

count = 0
def callback_graph(weights, obj_func_eval):
    global count
    count+=1
    print(count)

In [93]:
sampler = Sampler()
optimizer = COBYLA(maxiter=10)

In [94]:
%%time

vqc = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    callback=callback_graph,
)

# clear objective value history
objective_func_vals = []


vqc.fit(train_features_rbf, y_train.values.reshape(-1,1))

1
2
3
4
5
6
7
8
9
10
CPU times: user 9.64 s, sys: 75.9 ms, total: 9.72 s
Wall time: 13.1 s


In [95]:
test_score= vqc.score(test_features_rbf, y_test)
print(f"Quantum VQC on the test dataset:     {test_score:.2f}")

Quantum VQC on the test dataset:     0.63
